# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных
Данные геологоразведки трёх регионов находятся в файлах:

/datasets/geo_data_0.csv. Скачать датасет

/datasets/geo_data_1.csv. Скачать датасет

/datasets/geo_data_2.csv. Скачать датасет

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from IPython.display import display
import warnings
from scipy import stats

In [2]:
warnings.filterwarnings("ignore")

In [3]:
data0=pd.read_csv('/datasets/geo_data_0.csv')
display(data0.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data1=pd.read_csv('/datasets/geo_data_1.csv')
data1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data2=pd.read_csv('/datasets/geo_data_2.csv')
data2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
def fun_date(data):
    print('**Общая информация датафрейма**')
    print(data.info())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Просмотр статистических данных**')
    print(data.describe())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод рандомных строк датафрейма**')
    print(data.sample(n=3, random_state=1))
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод первых пяти строк датафрейма**')
    print(data.head())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод количесва дубликатов датафрейма**')
    print(data.duplicated().sum())
    print('-------------------------------------------------------------------------------------------------------------')
    print('**Вывод количесва пропущеных значений датафрейма**')
    print(data.isnull().sum())
    print('-------------------------------------------------------------------------------------------------------------')
print(fun_date(data0))
print(fun_date(data1))
print(fun_date(data2))

**Общая информация датафрейма**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-------------------------------------------------------------------------------------------------------------
**Просмотр статистических данных**
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581 

In [7]:
data0['id'].duplicated().sum()

10

In [8]:
data1['id'].duplicated().sum()

4

In [9]:
data2['id'].duplicated().sum()

4

In [10]:
# код ревьюера
print(data2.head())
display(data2.head())

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<font color='blue'>Все датафреймы выведены. Выведена вся информация по каждому датафрейму, данные готовы к работе.</font>

## Обучение и проверка модели

In [11]:
data0_train, data0_valid = train_test_split(data0, test_size=0.25, random_state=12345)
data1_train, data1_valid = train_test_split(data1, test_size=0.25, random_state=12345)
data2_train, data2_valid = train_test_split(data2, test_size=0.25, random_state=12345)


features_train0 = data0_train.drop(['id', 'product'], axis=1)
target_train0 = data0_train['product']
features_valid0 = data0_valid.drop(['id', 'product'], axis=1)
target_valid0 = data0_valid['product']

features_train1 = data1_train.drop(['id', 'product'], axis=1)
target_train1 = data1_train['product']
features_valid1 = data1_valid.drop(['id', 'product'], axis=1)
target_valid1 = data1_valid['product']

features_train2 = data2_train.drop(['id', 'product'], axis=1)
target_train2 = data2_train['product']
features_valid2 = data2_valid.drop(['id', 'product'], axis=1)
target_valid2 = data2_valid['product']

print('Обучающая выборка первого датафрейма:',features_train0.shape)
print(target_train0.shape)
print('Валидационная выборка первого датафрейма:',features_valid0.shape)
print(target_valid0.shape)
print()
print('Обучающая выборка второго датафрейма:',features_train1.shape)
print(target_train1.shape)
print('Валидационная выборка второго датафрейма:',features_valid1.shape)
print(target_valid1.shape)
print()
print('Обучающая выборка третьего датафрейма:',features_train2.shape)
print(target_train2.shape)
print('Валидационная выборка третьего датафрейма:',features_valid2.shape)
print(target_valid2.shape)

Обучающая выборка первого датафрейма: (75000, 3)
(75000,)
Валидационная выборка первого датафрейма: (25000, 3)
(25000,)

Обучающая выборка второго датафрейма: (75000, 3)
(75000,)
Валидационная выборка второго датафрейма: (25000, 3)
(25000,)

Обучающая выборка третьего датафрейма: (75000, 3)
(75000,)
Валидационная выборка третьего датафрейма: (25000, 3)
(25000,)


In [12]:
#Месторождение 1
model = LinearRegression()
model.fit(features_train0,target_train0) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid0) # предсказания модели на валидационной выборке
data0_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid0,predictions_valid)**0.5
data0_predictions_valid = model.predict(features_valid0)

print("Средний предсказанный запас предсказанного сырья в месторождении", data0_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid0.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний предсказанный запас предсказанного сырья в месторождении 92.59256778438038
Средний фактический запас сырья в месторождении 92.07859674082927
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


In [13]:
#Месторождение 2
model = LinearRegression()
model.fit(features_train1,target_train1) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid1) # предсказания модели на валидационной выборке
data1_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid1,predictions_valid)**0.5
data1_predictions_valid = model.predict(features_valid1)

print("Средний предсказанный запас предсказанного сырья в месторождении", data1_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid1.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний предсказанный запас предсказанного сырья в месторождении 68.728546895446
Средний фактический запас сырья в месторождении 68.72313602435997
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775616


In [14]:
#Месторождение 3
model = LinearRegression()
model.fit(features_train2,target_train2) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid2) # предсказания модели на валидационной выборке
data2_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid2,predictions_valid)**0.5
data2_predictions_valid = model.predict(features_valid2)

print("Средний предсказанный запас предсказанного сырья в месторождении", data2_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid2.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний предсказанный запас предсказанного сырья в месторождении 94.96504596800489
Средний фактический запас сырья в месторождении 94.88423280885438
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


<font color='blue'>Судя по показателю RSME самым выгодным явлеется второе месторождение, но оно очень уступает по запасам сырья в отличие от остальных.</font>

## Подготовка к расчёту прибыли

In [15]:
#Сколько денег выделяют на каждую скважину

MONEY_ONE_SKVAJ=10_000_000_000/200
print(MONEY_ONE_SKVAJ)
#сколько нужно тысяч баррелей для того, чтобы окупить затраты
COUNT_BARR=MONEY_ONE_SKVAJ/450000

print(COUNT_BARR)
data0_mean=92.07859674082927
data1_mean=68.72313602435997
data2_mean=94.88423280885438
print('Объём сырья для безубыточной разработки новой скважины 1 месторождения:',COUNT_BARR-data0_mean)
print('Объём сырья для безубыточной разработки новой скважины 2 месторождения:',COUNT_BARR-data1_mean)
print('Объём сырья для безубыточной разработки новой скважины 3 месторождения:',COUNT_BARR-data2_mean)

50000000.0
111.11111111111111
Объём сырья для безубыточной разработки новой скважины 1 месторождения: 19.032514370281845
Объём сырья для безубыточной разработки новой скважины 2 месторождения: 42.38797508675114
Объём сырья для безубыточной разработки новой скважины 3 месторождения: 16.226878302256736


<font color='blue'>Второе месторождение получается слишком затратным на первый взгляд, посмотрим как оно себя проявит позже
</font>

## Расчёт прибыли и рисков 

In [16]:
def profit(target, predictions, count):
    profit=0
    probs_sorted = predictions.sort_values(ascending=False).head(200)
    selected = target[probs_sorted.index][:count] 
    for top in selected:
        profit+=(top-COUNT_BARR)*450000
    return profit



In [17]:
data0_valid['product_pred']=data0_predictions_valid

profit0=profit(data0_valid['product'], data0_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit0/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 3321 млн.


In [18]:
data1_valid['product_pred']=data1_predictions_valid

profit1=profit(data1_valid['product'], data1_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit1/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2415 млн.


In [19]:
data2_valid['product_pred']=data2_predictions_valid

profit2=profit(data2_valid['product'], data2_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit2/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2710 млн.


In [20]:
def boots (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        
        values.append(profit(target_subsample, preds_subsample, 500))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print('Вероятность убытков',stats.percentileofscore(values, 0),'%')
    print("Среднее значение бутстрепа", (values.mean())/1000000, "млн.")
    print("Верхняя граница доверительного интервала", higher)
    print("Нижняя граница доверительного интервала", lower)
    print()
print("Для первого месторождения")
boots(data0_valid['product'], data0_valid['product_pred'])
print()
print("Для второго месторождения")
boots(data1_valid['product'], data1_valid['product_pred'])
print()
print("Для третьего месторождения")
boots(data2_valid['product'], data2_valid['product_pred'])









Для первого месторождения
Вероятность убытков 7.6 %
Среднее значение бутстрепа 403.535244261165 млн.
Верхняя граница доверительного интервала 931574367.5508462
Нижняя граница доверительного интервала -118327362.12311192


Для второго месторождения
Вероятность убытков 1.7 %
Среднее значение бутстрепа 465.04105822107704 млн.
Верхняя граница доверительного интервала 864388463.3254703
Нижняя граница доверительного интервала 32036742.850023583


Для третьего месторождения
Вероятность убытков 7.9 %
Среднее значение бутстрепа 414.2097228409675 млн.
Верхняя граница доверительного интервала 972339167.9202832
Нижняя граница доверительного интервала -148373898.68249193



Исходя из всего проведенного анализа, можно смело заявить, что второе месторождение самое выгодное, об этом говорит и показатель RMSE и процент убытков. Да, оно конечно слегка проседает по прибыльности с 200 скважен, но я думаю игра стоит свечь, хотябы можно быть увереным в стабильности дохода.